In [1]:
import datajoint as dj
import numpy as np
import os
import time
import trimesh

In [2]:
pinky = dj.create_virtual_module("pinky","microns_pinky")
schema = dj.schema("microns_pinky")

Connecting celiib@10.28.0.34:3306


In [ ]:
"""
Purpose: To build a new compartment component divider that accounts for disconnected parts 

Inputs that will be recieved: 
1) Mesh data
2) Labels of the mesh

Deisred Output:
1) Compartment Type
2) Compartment Index
3) Vertex Indicies of that group
4) Triangle Indices of that group


Pseudocode: 
1) Subdivide the mesh into different parts based on the labels
2) Subdivide the mesh into seperate components
3) Map back to original vertices

"""

In [ ]:
def filter_verts_and_faces(verts,faces,verts_label,triangles_label):
    verts = verts.tolist()
    faces = faces.tolist()
    verts_label = verts_label.tolist()
    triangles_label = triangles_label.tolist()
    
    verts_keep = []
    faces_keep = []
    verts_lookup = {}
    
    for i,ver in enumerate(verts_label):
        verts_keep.append(verts[ver])
        verts_lookup[ver] = i
    
    #generate the new face labels
    for fac in triangles_label:
        faces_with_verts = faces[fac]
        new_tuple = []
        for v in faces_with_verts:
            new_tuple.append(verts_lookup[v])
        
        faces_keep.append(new_tuple)
    #check that the new verts and faces to return are same length as the indices
    """if len(triangles_label) != len(faces_keep) or len(verts_label) != len(verts_keep):
        print("ERROR THE FILTERED LABELS ARE NOT THE SAME SIZE AS THE INDICES LISTS!")"""
     
    return verts_keep,faces_keep

In [ ]:
def load_datajoint_component(key):
    """
    Will import a component 
    """
    
    #add the clusters and smoothness
    key["clusters"] = 12
    key["smoothness"] = 0.04
    

    #neuron_ID = 579228
    #compartment_type = "Basal"
    #component_index = 2
    #clusters = 12
    #smoothness = 0.04
    
    #Apical_Basal_Oblique_default = [12,16]
    #basal_big = [16,18]
    
    neuron_ID = str(key["segment_id"])
    #component = (ta3.Compartment.Component & key).fetch1()

    component_index = key["component_index"]
    compartment_type = key["compartment_type"]
    #print("component_size = " + str(component_size))
    

    #get the mesh
    #places where to save
    
    #########--------------------  Importing the mesh component vertices and triangles----------  ##############
    mesh_file_location = "dendrite_branches"

    #download the mesh and the cgal files
    segment_type = compartment_type
    segment_id = neuron_ID
    component_index = component_index
    clusters=key["clusters"]
    smoothness=key["smoothness"]
    decimation_ratio=key["decimation_ratio"]
    
    print("Working on " + str(segment_id) + "_" + str(segment_type) + "_" + str(component_index))
    

    output_file_no_ext = str(segment_id) + "_" + str(segment_type) + "_" + str(component_index)

    #get the original mesh
    original_mesh_key = dict(segmentation=3,
                           segment_id=segment_id,
                            decimation_ratio= decimation_ratio
                           )
    
    type_neuron = ""
    
    if len(pinky.ExcitatoryLeftoverMeshes() & original_mesh_key) > 0:
        neuron_data = ((pinky.ExcitatoryLeftoverMeshes  & original_mesh_key).fetch(as_dict=True))[0]
        type_neuron = "excitatory"
    else:
        print("There is no table with that segment id")
        return 

    verts = neuron_data["vertices"]
    faces = neuron_data["triangles"]


    #get the indices of the component
    component_mesh_key = dict(compartment_type=segment_type,segmentation=3,
                           segment_id=segment_id,
                           component_index=component_index,
                            decimation_ratio= decimation_ratio
                           )
    
    if type_neuron == "excitatory":
        component_vertices, component_faces = (pinky.LeftoverCompartmentFinal.LeftoverComponentFinal & component_mesh_key).fetch1("vertex_indices", "triangle_indices")
    else:
        raise Exception("Component wasn't in excitatory or orphan compartments")
    
    print("Num of vertices = " + str(len(component_vertices)))
    print("Numb of faces = " + str(len(component_faces)))
    #use the mesh indices to get the submesh
    
    """
    new_mesh = trimesh.Trimesh()
    new_mesh.vertices = verts
    new_mesh.faces = faces

    #get the submesh according to the verties
    component_mesh = new_mesh.submesh([component_faces],append=True)

    #print the number of vertices to make sure they are the same
    print("Len original component vertices = " + str(len(component_vertices)))
    print("Len new component vertices = " + str(len(component_mesh.vertices)))
    
    verts = component_mesh.vertices.astype(dtype=np.int32).tolist()
    faces = component_mesh.faces.astype(dtype=np.uint32).tolist()
    """
    
    #New way of getting the vertices and faces for visualization
    print("about to enter filter verts")
    filtered_verts,filtered_faces = filter_verts_and_faces(verts,faces,component_vertices, component_faces)
    return filtered_verts,filtered_faces,
    

In [ ]:
segmentation=3
segment_id = 648518346341371119
decimation_ratio = 0.35
compartment_type = "Apical"
component_index = 0



key = dict(segmentation=segmentation,
            segment_id=segment_id,
            decimation_ratio=decimation_ratio,
            compartment_type=compartment_type,
            component_index=component_index)



original_start_time = time.time()    
filtered_verts,filtered_faces = load_datajoint_component(key)

#get the labels
face_labels,verts_labels = 




In [ ]:
new_mesh = trimesh.Trimesh(filtered_verts,filtered_faces,process=False)
new_mesh.show()

In [ ]:
numeric,description = pinky.LabelKey.fetch("numeric","description")
new_dict = dict(zip(numeric,description))
new_dict

In [ ]:
pinky.ExcitatoryLeftoverMeshes & "n_vertices>70000"

In [3]:
from trimesh.graph import connected_components

In [4]:
def celii_split(mesh,
          only_watertight=False,
          adjacency=None,
          engine=None):
    """
    Split a mesh into multiple meshes from face connectivity.
    If only_watertight is true, it will only return watertight meshes
    and will attempt single triangle/quad repairs.
    Parameters
    ----------
    mesh: Trimesh
    only_watertight: if True, only return watertight components
    adjacency: (n,2) list of face adjacency to override using the plain
               adjacency calculated automatically.
    engine: str, which engine to use. ('networkx', 'scipy', or 'graphtool')
    Returns
    ----------
    meshes: list of Trimesh objects
    """
    
    
    if adjacency is None:
        adjacency = mesh.face_adjacency

    # if only watertight the shortest thing we can split has 3 triangles
    if only_watertight:
        min_len = 3
    else:
        min_len = 1

    components = connected_components(edges=adjacency,
                                      nodes=np.arange(len(mesh.faces)),
                                      min_len=min_len,
                                      engine=engine)
    return components

In [5]:
def get_vertex_indices(split_face_indices,total_mesh):
    """
    Will generate the vertex indices from the original face
    based on the faces indices passed along with the original mesh
    (can accept just one face indices arrray or list of arrays)
    
    Pseudocode: 
    1) Get the original faces array
    2) Get a list of all of the vertices indexes from the face indices
    3) get a unique list and return it
    """
    
    vertex_indices = []

    original_face_list = np.array(list(total_mesh.faces))

    for face_indices in split_face_indices:
        #print(face_indices)
        #get the corresponding vertices indexes
        #print(original_face_list[face_indices].ravel())
        unique_vert_indexes = np.unique(original_face_list[face_indices].ravel())
        #print(unique_vert_indexes)
        vertex_indices.append(unique_vert_indexes)

    return vertex_indices
        
    

ValueError: too many values to unpack (expected 2)

In [20]:
def make_datajoint_keys(key,label_name,vertex_indices,face_indexes_real):
    #make sure they both have the same length
    print(f"len(vertex_indices) = " + str(len(vertex_indices)))
    print(f"len(face_indexes_real) = " + str(len(face_indexes_real)))
    
    if len(face_indexes_real) != len(vertex_indices):
        raise Exception("Length of vertex indices and face indices lists do not match in make datajoint keys")
    
    #print(f"face_indexes_real = {face_indexes_real}")
    #print(f"vertex_indices = {vertex_indices}")
    key_list = []
    for j in range(0,len(face_indexes_real)):
        vertex_list = vertex_indices[j]
        face_list = face_indexes_real[j]
        
        current_key = dict(key,
                           compartment_type = label_name,
                           component_index=j,
                           n_vertex_indices = len(vertex_list),
                           n_triangle_indices = len(face_list),
                           vertex_indices = vertex_list,
                           triangle_indices = face_list
                          )
        key_list.append(current_key)
    return key_list
        

In [21]:
"""
Pseudocdoe for trimesh version
1) Retrieve full mesh
2) Retrieve full labels
3) For each unique label
a) Use trimesh to subdivide the mesh into each of the sepearate compartments
b) Use the split method to divide into distinct bodies
c) Save all of those as keys

"""



key = dict(segment_id=648518346349496405,segmentation=3)

numeric,description = pinky.LabelKey.fetch("numeric","description")
labels_dict = dict(zip(numeric,description))

#get the full mesh
verts,faces = (pinky.ExcitatoryLeftoverMeshes & key).fetch1("vertices","triangles")

#get the labels
verts_labels,faces_labels = (pinky.LeftoverCoarseLabelFinal & key).fetch1("vertices","triangles")
verts_labels

#get the number of unique labels:
unique_labels = np.unique(verts_labels)
print(unique_labels)

#create the Trimesh of the whole mesh
total_mesh = trimesh.Trimesh(verts,faces,process=False)



#iterate through all of the different labels
total_keys = []
for label in unique_labels:
    print("Working on label " + str(label))
    face_indexes = np.where(faces_labels==label)[0]
    #create a submesh
    new_mesh = total_mesh.submesh([face_indexes],only_watertight=False,append=False)[0]#.split()
    split_face_indices = celii_split(new_mesh)
    
    #print(f"faces_indexes = {face_indexes}")
    #print(f"split_face_indices = {np.array(split_face_indices)}")
    
    face_indexes_real = []
    for sp in split_face_indices:
        face_indexes_real.append(face_indexes[sp])
    
    #get the real indices 
    #face_indexes_real = face_indexes[np.array(split_face_indices)]
    
    vertex_indices = get_vertex_indices(face_indexes_real,total_mesh)
    
    dj_keys = make_datajoint_keys(key,labels_dict[label],vertex_indices,face_indexes_real)
    
    total_keys = total_keys + dj_keys

#then need to insert the total keys into datajoint
print(len(total_keys))
    
    



[2 3 5 6]
Working on label 2
len(vertex_indices) = 21
len(face_indexes_real) = 21
Working on label 3
len(vertex_indices) = 36
len(face_indexes_real) = 36
Working on label 5
len(vertex_indices) = 17
len(face_indexes_real) = 17
Working on label 6
len(vertex_indices) = 4
len(face_indexes_real) = 4
78


In [35]:
pinky.LeftoverCompartmentFinal()#.LeftoverComponentFinal()

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio
3,648518346341371119,0.35
3,648518346349386137,0.35
3,648518346349470171,0.35
3,648518346349471156,0.35
3,648518346349471500,0.35
3,648518346349471562,0.35
3,648518346349471565,0.35


In [22]:
total_keys[0]

{'segment_id': 648518346349496405,
 'segmentation': 3,
 'compartment_type': 'Apical',
 'component_index': 0,
 'n_vertex_indices': 17008,
 'n_triangle_indices': 33968,
 'vertex_indices': array([196817, 196818, 196819, ..., 213822, 213823, 213824]),
 'triangle_indices': array([394705, 417358, 417357, ..., 406026, 406027, 406028])}

In [25]:
def filter_verts_and_faces(verts,faces,verts_label,triangles_label):
    verts = verts.tolist()
    faces = faces.tolist()
    verts_label = verts_label.tolist()
    triangles_label = triangles_label.tolist()
    
    verts_keep = []
    faces_keep = []
    verts_lookup = {}
    
    for i,ver in enumerate(verts_label):
        verts_keep.append(verts[ver])
        verts_lookup[ver] = i
    
    #generate the new face labels
    for fac in triangles_label:
        faces_with_verts = faces[fac]
        new_tuple = []
        for v in faces_with_verts:
            new_tuple.append(verts_lookup[v])
        
        faces_keep.append(new_tuple)
    #check that the new verts and faces to return are same length as the indices
    """if len(triangles_label) != len(faces_keep) or len(verts_label) != len(verts_keep):
        print("ERROR THE FILTERED LABELS ARE NOT THE SAME SIZE AS THE INDICES LISTS!")"""
     
    return verts_keep,faces_keep

In [32]:
#testing if correct mesh vertices are created
example_key = total_keys[3]
vertex_indices,triangle_indices = example_key["vertex_indices"],example_key["triangle_indices"]

verts_keep,faces_keep = filter_verts_and_faces(verts,faces,vertex_indices,triangle_indices)

test_mesh = trimesh.Trimesh(verts_keep,faces_keep,process=False)
test_mesh.show()

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [ ]:
""